In [1]:
import csv
import pandas as pd
import numpy as np
import sklearn.metrics
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder
from collections import Counter
from datetime import datetime
from pathlib import Path
from pprint import pprint
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, precision_recall_curve,roc_curve,log_loss,balanced_accuracy_score
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay

from pycaret.classification import *

In [86]:
# 数据目录
root = Path('/data/ab01/standard_dataset/processed_dataset')
train_data_paths = [root/f'CM00{i}'/f'CM00{i}_train_clone-pass_germ-pass.tsv' for i in [0,1,2,3]]
valid_data_paths = [root/f'CM00{i}'/f'CM00{i}_valid_clone-pass.tsv' for i in [1,2,3]]

train_data_paths

[PosixPath('/data/ab01/standard_dataset/processed_dataset/CM000/CM000_train_clone-pass_germ-pass.tsv'),
 PosixPath('/data/ab01/standard_dataset/processed_dataset/CM001/CM001_train_clone-pass_germ-pass.tsv'),
 PosixPath('/data/ab01/standard_dataset/processed_dataset/CM002/CM002_train_clone-pass_germ-pass.tsv'),
 PosixPath('/data/ab01/standard_dataset/processed_dataset/CM003/CM003_train_clone-pass_germ-pass.tsv')]

In [87]:
valid_data_paths

[PosixPath('/data/ab01/standard_dataset/processed_dataset/CM001/CM001_valid_clone-pass.tsv'),
 PosixPath('/data/ab01/standard_dataset/processed_dataset/CM002/CM002_valid_clone-pass.tsv'),
 PosixPath('/data/ab01/standard_dataset/processed_dataset/CM003/CM003_valid_clone-pass.tsv')]

## clonal lineage
> Reference: https://www.pnas.org/cgi/doi/10.1073/pnas.1814213116)
### step1
group sequences having the same V and J germline genes and CDR3 length

### setp2
<!-- Within each group, performing single-linkage clustering on the CDR3 sequence using a cutoff of 90% sequence identity -->

In [ ]:
aa-cdist: 每一条序列到该进化树共有序列(consensus sequence)的距离，通过hamming distance进行测量——数值越小表示亲和力越强。间接反应了我们之前观测到的特异性clonotype里，末端进化支长度普遍较短。
dist_mrca: 0.03358314574722221
leaf_bl: 0.026341668088888884
nuc-lbi: 0.006910531404293908
nuc-lbr: 内部节点到其所有外部节点的进化支总长度÷该节点到进化树的根的进化支总长度。该参数可用来定量一个增强亲和力的变异在这个节点产生的概率，因为该变异会增强该节点后代的适应度——所以该数值越大表示其适应度越强。
sackin_idx: 10.527777777777779

In [96]:
for file in train_data_paths:
    df0 = pd.read_csv(file, sep='\t')
    df0['cdr3_seq_len'] = df0['cdr3'].apply(lambda x: len(x))
    df0['junction_seq_len'] = df0['cdr3'].apply(lambda x: len(x))
    # df0['germline_v_call'] = df0['v_call'].apply(lambda x: x.split(',')[0] if x else None)
    # df0['germline_j_call'] = df0['j_call'].apply(lambda x: x.split(',')[0] if x else None)
    df0['v_gene_name'] = df0['germline_v_call'].apply(lambda x: x.split('*')[0] if x else None)
    df0['j_gene_name'] = df0['germline_j_call'].apply(lambda x: x.split('*')[0] if x else None)
    
    # grouped = df0.groupby(by=['v_germline_alignment', 'j_germline_alignment', 'cdr3_len'])
    # grouped2 = df0.groupby(by=['v_gene_name', 'j_gene_name', 'cdr3_len'])
    # df0['group_id_by_germline_seq'] = None
    
    # group_id = 0
    # for key, idx in grouped2.groups.items():
    #     group_id+=1
    #     df0.loc[idx, 'group_id_by_germline_seq'] = group_id
    
    df0 = df0.where(pd.notnull(df0), None)
    with open(f'data/test/{file.name}', 'w') as f:
        f_csv = csv.writer(f, delimiter='\t')
        f_csv.writerow(df0.columns)
        f_csv.writerows(df0.values)

In [97]:
for file in valid_data_paths:
    df0 = pd.read_csv(file, sep='\t')
    df0['cdr3_seq_len'] = df0['cdr3'].apply(lambda x: len(x))
    df0['junction_seq_len'] = df0['cdr3'].apply(lambda x: len(x))
    df0['germline_v_call'] = df0['v_call'].apply(lambda x: x.split(',')[0] if x else None)
    df0['germline_j_call'] = df0['j_call'].apply(lambda x: x.split(',')[0] if x else None)
    df0['v_gene_name'] = df0['germline_v_call'].apply(lambda x: x.split('*')[0] if x else None)
    df0['j_gene_name'] = df0['germline_j_call'].apply(lambda x: x.split('*')[0] if x else None)
    
    # grouped = df0.groupby(by=['v_germline_alignment', 'j_germline_alignment', 'cdr3_len'])
#     grouped2 = df0.groupby(by=['germline_v_call_without_allele', 'germline_j_call_without_allele', 'cdr3_len'])
#     df0['group_id_by_germline_seq'] = None
    
#     group_id = 0
#     for key, idx in grouped2.groups.items():
#         group_id+=1
#         df0.loc[idx, 'group_id_by_germline_seq'] = group_id
    
    df0 = df0.where(pd.notnull(df0), None)
    with open(f'data/test/{file.name}', 'w') as f:
        f_csv = csv.writer(f, delimiter='\t')
        f_csv.writerow(df0.columns)
        f_csv.writerows(df0.values)

In [132]:
df1_t = pd.read_csv('data/test/CM001_train_clone-pass_germ-pass.tsv', sep='\t')
df1_v = pd.read_csv('data/test/CM001_valid_clone-pass.tsv', sep='\t')
df1_all = pd.concat([df1_t.loc[:, ['sequence_id', 'cdr3_seq_len', 'v_gene_name', 'j_gene_name', 'clone_id']], df1_v.loc[:, ['sequence_id', 'cdr3_seq_len', 'v_gene_name', 'j_gene_name', 'clone_id']]], axis=0)
# df1_all['clone_id_by_same_cdr3_len'] = None
df1_all['clone_id_by_same_gene_name'] = None

grouped = df1_all.groupby(by=['v_gene_name', 'j_gene_name', 'cdr3_seq_len'])
group_id = 0
for key, idx in grouped.groups.items():
    group_id+=1
    df1_all.loc[idx, 'clone_id_by_same_gene_name'] = group_id
    
df1_t = pd.merge(df1_t, df1_all.loc[:, ['sequence_id','clone_id_by_same_gene_name']], on='sequence_id', how='left')
df1_v = pd.merge(df1_v, df1_all.loc[:, ['sequence_id','clone_id_by_same_gene_name']], on='sequence_id', how='left')

df1_t = df1_t.where(pd.notnull(df1_t), None)
with open('data/cluster_by_same_gene_name/CM001_train_clone-pass_germ-pass.tsv', 'w') as f:
    f_csv = csv.writer(f, delimiter='\t')
    f_csv.writerow(df1_t.columns)
    f_csv.writerows(df1_t.values)

df1_v = df1_v.where(pd.notnull(df1_v), None)
with open('data/cluster_by_same_gene_name/CM001_valid_clone-pass.tsv', 'w') as f:
    f_csv = csv.writer(f, delimiter='\t')
    f_csv.writerow(df1_v.columns)
    f_csv.writerows(df1_v.values)

In [131]:
df1_v['clone_id_by_same_gene_name'].isnull().sum()

0

In [133]:
df2_t = pd.read_csv('data/test/CM002_train_clone-pass_germ-pass.tsv', sep='\t')
df2_v = pd.read_csv('data/test/CM002_valid_clone-pass.tsv', sep='\t')
df2_all = pd.concat([df2_t.loc[:, ['sequence_id', 'cdr3_seq_len', 'v_gene_name', 'j_gene_name', 'clone_id']], df2_v.loc[:, ['sequence_id', 'cdr3_seq_len', 'v_gene_name', 'j_gene_name', 'clone_id']]], axis=0)
# df2_all['clone_id_by_same_cdr3_len'] = None
df2_all['clone_id_by_same_gene_name'] = None

grouped = df2_all.groupby(by=['v_gene_name', 'j_gene_name', 'cdr3_seq_len'])
group_id = 0
for key, idx in grouped.groups.items():
    group_id+=2
    df2_all.loc[idx, 'clone_id_by_same_gene_name'] = group_id
    
df2_t = pd.merge(df2_t, df2_all.loc[:, ['sequence_id','clone_id_by_same_gene_name']], on='sequence_id', how='left')
df2_v = pd.merge(df2_v, df2_all.loc[:, ['sequence_id','clone_id_by_same_gene_name']], on='sequence_id', how='left')

df2_t = df2_t.where(pd.notnull(df2_t), None)
with open('data/cluster_by_same_gene_name/CM002_train_clone-pass_germ-pass.tsv', 'w') as f:
    f_csv = csv.writer(f, delimiter='\t')
    f_csv.writerow(df2_t.columns)
    f_csv.writerows(df2_t.values)

df2_v = df2_v.where(pd.notnull(df2_v), None)
with open('data/cluster_by_same_gene_name/CM002_valid_clone-pass.tsv', 'w') as f:
    f_csv = csv.writer(f, delimiter='\t')
    f_csv.writerow(df2_v.columns)
    f_csv.writerows(df2_v.values)

In [134]:
df3_t = pd.read_csv('data/test/CM003_train_clone-pass_germ-pass.tsv', sep='\t')
df3_v = pd.read_csv('data/test/CM003_valid_clone-pass.tsv', sep='\t')
df3_all = pd.concat([df3_t.loc[:, ['sequence_id', 'cdr3_seq_len', 'v_gene_name', 'j_gene_name', 'clone_id']], df3_v.loc[:, ['sequence_id', 'cdr3_seq_len', 'v_gene_name', 'j_gene_name', 'clone_id']]], axis=0)
# df3_all['clone_id_by_same_cdr3_len'] = None
df3_all['clone_id_by_same_gene_name'] = None

grouped = df3_all.groupby(by=['v_gene_name', 'j_gene_name', 'cdr3_seq_len'])
group_id = 0
for key, idx in grouped.groups.items():
    group_id+=3
    df3_all.loc[idx, 'clone_id_by_same_gene_name'] = group_id
    
df3_t = pd.merge(df3_t, df3_all.loc[:, ['sequence_id','clone_id_by_same_gene_name']], on='sequence_id', how='left')
df3_v = pd.merge(df3_v, df3_all.loc[:, ['sequence_id','clone_id_by_same_gene_name']], on='sequence_id', how='left')

df3_t = df3_t.where(pd.notnull(df3_t), None)
with open('data/cluster_by_same_gene_name/CM003_train_clone-pass_germ-pass.tsv', 'w') as f:
    f_csv = csv.writer(f, delimiter='\t')
    f_csv.writerow(df3_t.columns)
    f_csv.writerows(df3_t.values)

df3_v = df3_v.where(pd.notnull(df3_v), None)
with open('data/cluster_by_same_gene_name/CM003_valid_clone-pass.tsv', 'w') as f:
    f_csv = csv.writer(f, delimiter='\t')
    f_csv.writerow(df3_v.columns)
    f_csv.writerows(df3_v.values)

In [144]:
df3_all['clone_id'].value_counts()

2059    1482
2023     616
1333     598
928      458
2081     342
        ... 
235        1
234        1
1525       1
232        1
1988       1
Name: clone_id, Length: 2186, dtype: int64

In [143]:
df3_all['clone_id_by_same_gene_name'].value_counts()

948     1491
2418     710
2871     629
354      616
1443     484
        ... 
3          1
651        1
444        1
867        1
1581       1
Name: clone_id_by_same_gene_name, Length: 924, dtype: int64

In [145]:
df0_t = pd.read_csv('data/test/CM000_train_clone-pass_germ-pass.tsv', sep='\t')
df0_t['clone_id_by_same_gene_name'] = None

grouped = df0_t.groupby(by=['v_gene_name', 'j_gene_name', 'cdr3_seq_len'])
group_id = 0
for key, idx in grouped.groups.items():
    group_id+=3
    df0_t.loc[idx, 'clone_id_by_same_gene_name'] = group_id

df0_t = df0_t.where(pd.notnull(df0_t), None)
with open('data/cluster_by_same_gene_name/CM000_train_clone-pass_germ-pass.tsv', 'w') as f:
    f_csv = csv.writer(f, delimiter='\t')
    f_csv.writerow(df0_t.columns)
    f_csv.writerows(df0_t.values)

In [151]:
df0_t['clone_id'].value_counts()

383    35
403    32
577    30
517    27
625    23
       ..
411     1
414     1
416     1
417     1
99      1
Name: clone_id, Length: 876, dtype: int64

In [150]:
df0_t['clone_id_by_same_gene_name'].value_counts()

1113    39
1179    38
72      37
1194    33
51      33
        ..
387      1
27       1
483      1
162      1
270      1
Name: clone_id_by_same_gene_name, Length: 511, dtype: int64

In [ ]:
# 计算identity矩阵
# for V gene allel


# for J gene allel
